<a href="https://colab.research.google.com/github/ShiyuGong/AutoML-Vision-of-Flower-Dataset/blob/master/AutoML_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Overview ***

AutoML Vision enables us to train machine learning models to classify images according to our own defined labels. For this project, we will utilize free tier AutoML to train our own model of the flower dataset (the publicly available dataset of flower images from gs://cloud-ml-data/img/flower_photos/), evaluate the model performance and then predict two flower pictures with Vision API.


## Configure project environment

1. In the GCP console, go to the Manage resources page and select or create a project, 
2. then make sure that billing is enabled for the Google Cloud Platform project and enable the AutoML Vision APIs. 
3. After that, a service account needs to be created and a key file should be downloaded and stored in the Google Drive. 
4. In the following steps, set the GOOGLE_APPLICATION_CREDENTIALS environment variable to the path to the service account key file that we downloaded when we created the service account. 
5. Add the new service account to the AutoML Editor IAM role to allow the AutoML Vision service accounts to access the Google Cloud project resources. 
6.Then, set the PROJECT_ID and REGION_NAME environment variables. 
7.Create a Google Cloud Storage bucket to store the documents that we will use to train your custom model. 
8.Copy the publicly available dataset of flower images into our Google Cloud Storage bucket.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import os;os.listdir("/content/gdrive/My Drive/awsml")

['config.txt', 'credentials.txt', 'cloudai-16c0b9420d70.json']

In [0]:
!ls -la /content/gdrive/My\ Drive/awsml

total 4
-rw------- 1 root root 2311 Mar  5 02:18 cloudai-16c0b9420d70.json
-rw------- 1 root root   28 Jan  9 04:01 config.txt
-rw------- 1 root root  113 Jan  9 04:26 credentials.txt


In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/gdrive/My\ Drive/awsml/cloudai-16c0b9420d70.json"
!gcloud auth activate-service-account --key-file /content/gdrive/My\ Drive/awsml/cloudai-16c0b9420d70.json
!pip install --upgrade -q google-cloud-vision

Activated service account credentials for: [cloudai@cloudai-233600.iam.gserviceaccount.com]


## AutoML Vision Trained Model of Flower Dataset

After 3667 flower images are imported, we could use free tier of AutoML to train our model. The evaluation results provided are as follows. The average precision is 0.996; Based on a score threshold of 0.5, Precision is 98.466% and Recall is 96.396%. (A high precision model produces fewer false positives, and a high recall model produces fewer false negatives.) Overall, the AutoML Vision trained model of flower dataset has a really good performance.

![automl_model1](https://docs.google.com/uc?export=download&id=1ZiN_CQt1i7nw7b0ndO5fefg7ZgvMAQw7)
![automl_model2](https://docs.google.com/uc?export=download&id=1MrE3-1GkywNTYbVfcrnGTtjE8RIj2iFB)



## Predict with Vision API 

In this step, we will go further to predict two flower pictures (one is daisy and the other is sunflower) with Vision API. After setting the project id, compute region, model id, file name and score threshold, we could get the full path of the model and create client for prediction service and then read the images and assign to payload. 

The predicted results for both images are really good. The predicted class names are correct with high predicted class scores, which are 0.99986732006073 and 0.9999873638153076 respectively for daisy and sunflower.


In [0]:
os.listdir("/content/gdrive/My Drive/MachineLearning/AutoML")

['sunflower.jpg', 'daisy.jpg']

![automl_model1](https://docs.google.com/uc?export=download&id=1GLe31TPEt8bz40RUyP3KGXH07kKHkFpb)
![automl_model2](https://docs.google.com/uc?export=download&id=1wptx5u84y-jB5PM9WGYH2ZCUObxKX6bq)


In [0]:
!pip uninstall gcloud google-cloud
!pip install --upgrade google-cloud-automl

In [0]:
project_id = 'cloudai-233600'
compute_region = 'us-central1'
model_id = 'ICN6100566313772959995'
file_name = '/content/gdrive/My Drive/MachineLearning/AutoML/daisy.jpg'
score_threshold = '0.5'

from google.cloud import automl_v1beta1 as automl

automl_client = automl.AutoMlClient()

# Get the full path of the model.
model_full_id = automl_client.model_path(
    project_id, compute_region, model_id
)

# Create client for prediction service.
prediction_client = automl.PredictionServiceClient()

# Read the image and assign to payload.
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

payload = {"image": {"image_bytes": content}}

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
# Initialize params
params = {}
if score_threshold:
    params = {"score_threshold": score_threshold}

response = prediction_client.predict(model_full_id, payload, params)
print("Prediction results of daisy.jpg:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print("Predicted class score: {}".format(result.classification.score))
    

Prediction results of daisy.jpg:
Predicted class name: daisy
Predicted class score: 0.99986732006073


In [0]:
file_name2 = '/content/gdrive/My Drive/MachineLearning/AutoML/sunflower.jpg'

# Read the image and assign to payload.
with io.open(file_name2, 'rb') as image_file:
    content = image_file.read()

payload = {"image": {"image_bytes": content}}

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
# Initialize params
params = {}
if score_threshold:
    params = {"score_threshold": score_threshold}

response = prediction_client.predict(model_full_id, payload, params)
print("Prediction results of sunflower.jpg:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print("Predicted class score: {}".format(result.classification.score))
    

Prediction results of sunflower.jpg:
Predicted class name: sunflowers
Predicted class score: 0.9999873638153076


## Conclusion

AutoML Vision, a graphical drag-and-drop tool that lets users leverage Google’s cloud computing backend to train self-learning object recognition and image detection models, is really useful for data analysis, which saves us time to train the model by ourselves and provides high-performance models. What we need to do is only to learn how to call APIs. 